<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##試算表個人記帳
- 可自行加入日期時間或按enter輸入現在時間。
- 輸入品項後可選擇屬於哪個種類，用於分析圖表。
- 可計算出當日總額即當月總額。
- [試算表連結](https://docs.google.com/spreadsheets/d/14qT33yoGpMBe868XI6yvd8Hy7DldM4RhovGYNRtCUMk/edit?hl=zh-tw&gid=0#gid=0)

In [150]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [151]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14qT33yoGpMBe868XI6yvd8Hy7DldM4RhovGYNRtCUMk/edit?hl=zh-tw&gid=0#gid=0')

In [152]:
#當日總支出fun
def update_daily_total(df, worksheet, today_str):
    """
    更新 Google Sheet 中的每日總支出
    df          : pandas DataFrame，至少需要欄位 ["日期_str", "金額"]
    worksheet   : gspread 的 worksheet 物件
    today_str   : 字串格式的日期 (例如 "2025-09-12")
    """

In [153]:
worksheet = gsheets.worksheet("工作表1")
headers = ["日期", "時間", "品項", "種類", "金額"]
first_row = worksheet.row_values(1)
if not first_row or first_row != headers:
    worksheet.clear()
    worksheet.append_row(headers)

In [154]:

# === 輸入資料 ===
date_str = input("請輸入日期 (格式：YYYY-MM-DD, Enter=今天): ")
if date_str.strip() == "":
    date_str = datetime.now().strftime("%Y-%m-%d")
else:
    datetime.strptime(date_str, "%Y-%m-%d")

time_str = input("請輸入時間 (格式：HH:MM, Enter=現在時間): ")
if time_str.strip() == "":
    time_str = datetime.now().strftime("%H:%M")
else:
    datetime.strptime(time_str, "%H:%M")

item = input("請輸入品項: ")
amount = float(input("請輸入金額: "))

categories = ["早餐", "午餐", "晚餐", "點心", "飲料", "衣服", "娛樂", "交通"]
print("請選擇種類：")
for i, c in enumerate(categories, 1):
    print(f"{i}. {c}")
choice = int(input("輸入數字選擇: "))
category = categories[choice - 1]

worksheet.append_row([date_str, time_str, item, category, amount], value_input_option="USER_ENTERED")
print("✅ 新增成功！")

請輸入日期 (格式：YYYY-MM-DD, Enter=今天): 
請輸入時間 (格式：HH:MM, Enter=現在時間): 
請輸入品項: 麵包
請輸入金額: 200
請選擇種類：
1. 早餐
2. 午餐
3. 晚餐
4. 點心
5. 飲料
6. 衣服
7. 娛樂
8. 交通
輸入數字選擇: 2
✅ 新增成功！


In [157]:
# === 轉 DataFrame 處理 ===
sheets = worksheet.get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df["金額"] = df["金額"].replace(r"[^\d.]+", "", regex=True).astype(float)
df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")

today_str = datetime.now().strftime("%Y-%m-%d")


ValueError: could not convert string to float: ''

In [158]:
    # 計算當日總支出
    today_total = round(df[df["日期_str"] == today_str]["金額"].sum(), 2)

    # 找到今天日期的所有列
    rows = worksheet.findall(today_str)
    last_row_today = rows[-1].row if rows else worksheet.row_count + 1

    # 檢查下一列是否已有「當天總支出」
    next_row_values = worksheet.row_values(last_row_today + 1)
    if next_row_values and next_row_values[0] == today_str and "當天總支出" in next_row_values:
        worksheet.update_cell(last_row_today + 1, 5, float(today_total))
    else:
        worksheet.insert_row([today_str, "", "當天總支出", "", float(today_total)], last_row_today + 1)

    # 👉 列印當日總花費
    print(f"📌 {today_str} 當日總支出：{today_total} 元")


KeyError: '日期_str'

In [147]:
# === 每月總結 ===
year = int(date_str[:4])
month = int(date_str[5:7])
month_total = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]["金額"].sum()
print(f"📌 {date_str[:7]} 當月總支出：{month_total} 元")

📌 2025-09 當月總支出：300.0 元


In [149]:
from datetime import datetime
import calendar

# === 更新圓餅圖 (隨時更新) ===
def update_pie_chart(df, worksheet, year, month):
    month_data = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]
    cat_summary = month_data.groupby("種類")["金額"].sum().reset_index()

    # 寫入 G 欄
    worksheet.update("G1", [["種類", "總金額"]])
    if not cat_summary.empty:
        worksheet.update("G2", cat_summary.values.tolist())

    # 建立圓餅圖 (會自動覆蓋舊的)
    chart_request = {
        "addChart": {
            "chart": {
                "spec": {
                    "title": f"{year}-{month:02d} 各種類支出分布",
                    "pieChart": {
                        "legendPosition": "RIGHT_LEGEND",
                        "domain": {
                            "sourceRange": {
                                "sources": [{"sheetId": worksheet.id,
                                             "startRowIndex": 0,
                                             "endRowIndex": len(cat_summary)+1,
                                             "startColumnIndex": 6,  # G
                                             "endColumnIndex": 7}]
                            }
                        },
                        "series": {
                            "sourceRange": {
                                "sources": [{"sheetId": worksheet.id,
                                             "startRowIndex": 0,
                                             "endRowIndex": len(cat_summary)+1,
                                             "startColumnIndex": 7,  # H
                                             "endColumnIndex": 8}]
                            }
                        },
                        "pieHole": 0.3  # 甜甜圈圖
                    }
                },
                "position": {
                    "overlayPosition": {
                        "anchorCell": {"sheetId": worksheet.id, "rowIndex": 0, "columnIndex": 9}
                    }
                }
            }
        }
    }
    gsheets.batch_update({"requests": [chart_request]})
    print("✅ 圓餅圖已更新")


In [138]:
# === 每次輸入資料後都更新圓餅圖 ===
year = int(date_str[:4])
month = int(date_str[5:7])
update_pie_chart(df, worksheet, year, month)


/tmp/ipython-input-2448527816.py:10: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("G1", [["種類", "總金額"]])
/tmp/ipython-input-2448527816.py:12: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("G2", cat_summary.values.tolist())


✅ 圓餅圖已更新
